In [1]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import pandas_ta as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

key = os.environ['av_key']

In [2]:
cc = CryptoCurrencies(key=key, output_format='pandas')

In [3]:
df, meta_data = cc.get_crypto_intraday('ETH', market='USD', interval='1min', outputsize='full')

In [4]:
df = df.rename(columns={"4. close": "close",
                   "1. open": "open",
                   "2. high": "high",
                   "3. low": "low",
                   "5. volume": "volume"})

df = df[['open', 'high', 'low', 'close', 'volume']]

In [5]:
df = df.sort_index()
df['idx_int'] = np.arange(0, len(df))
df = df.reset_index()

In [6]:
df.head()

,date,open,high,low,close,volume,idx_int
0,2024-11-08 10:53:00,2920.67,2922.81,2920.65,2922.79,29.0,0
1,2024-11-08 10:54:00,2922.48,2924.93,2922.38,2924.21,51.0,1
2,2024-11-08 10:55:00,2924.17,2925.30,2923.91,2925.17,31.0,2
3,2024-11-08 10:56:00,2925.18,2930.20,2925.18,2928.75,89.0,3
4,2024-11-08 10:57:00,2928.69,2934.19,2927.92,2932.64,221.0,4


In [7]:
def compute_gradient(start_index, df, x_label, y_label, len_data):
    # Ensure we only take data points from n to n+5
    if start_index + len_data > len(df):
        return None  # Return None if there are not enough points to calculate gradient
    
    # Extract the last 5 data points (x, y) from the DataFrame
    data_segment = df.iloc[start_index:start_index + len_data]
    x = data_segment[x_label]
    y = data_segment[y_label]
    
    # Calculate the necessary summations for the least squares formula
    n = len(x)
    sum_x = x.sum()
    sum_y = y.sum()
    sum_x2 = (x**2).sum()
    sum_xy = (x * y).sum()
    
    # Calculate the slope (gradient) using the least squares formula
    slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2)
    return slope

In [8]:
# realtime stock trading
df.ta.ema(length=10, append=True)
df.ta.ema(length=50, append=True)
df.ta.rsi(length=14, append=True)
# df.ta.rsi(length=12, append=True)
# df.ta.rsi(length=24, append=True)
df.ta.bbands(length=20, std=2, append=True)
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df.ta.psar(append=True)

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
0,NaN,NaN,0.02,0
1,2914.930000,NaN,0.04,0
2,2915.330000,NaN,0.06,0
3,2915.928200,NaN,0.08,0
4,2917.069944,NaN,0.10,0
...,...,...,...,...
345,2911.680000,NaN,0.02,1
346,2911.879200,NaN,0.02,0
347,2912.074416,NaN,0.02,0
348,2912.265728,NaN,0.02,0


In [9]:
df['psar_flip_dir'] = 0
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARl_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = 1
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARs_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = -1

In [10]:
df['r_ema_s_m'] = df['EMA_9'] / df['EMA_21']

In [11]:
len_data = 5

# Loop through the DataFrame, compute gradient for each row (starting point)
for i in range(len(df) - len_data):  # Make sure we have at least 5 points for each calculation
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_9', len_data)
    df.at[i + len_data-1, 'gradient_ema_9'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'RSI_7', len_data)
    df.at[i + len_data-1, 'gradient_rsi_7'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'close', len_data)
    df.at[i + len_data-1, 'gradient_ls'] = gradient

In [24]:
mask_crossing = (df['r_ema_s_m']<1.0007)&(df['r_ema_s_m']>0.9993)
df['flag_ema_crossing'] = 0
df.loc[mask_crossing, 'flag_ema_crossing'] = 1

In [25]:
mask_ema_grad_pos = (df['gradient_ema_9']>0.05)
mask_ema_grad_neg = (df['gradient_ema_9']<-0.05)
df['flag_grad_ema'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema'] = -1

mask_rsi_grad_pos = (df['gradient_rsi_7']>=1)
mask_rsi_grad_neg = (df['gradient_rsi_7']<=1)
df['flag_grad_rsi'] = 0
df.loc[mask_rsi_grad_pos, 'flag_grad_rsi'] = 1
df.loc[mask_rsi_grad_neg, 'flag_grad_rsi'] = -1

df['flag_grad_ls'] = 0
df.loc[df['gradient_ls'] >= 0.05, 'flag_grad_ls'] = 1
df.loc[df['gradient_ls'] <= -0.05, 'flag_grad_ls'] = -1

In [26]:
# df.ta.rsi(length=6, append=True)
# df.ta.rsi(length=12, append=True)
# df.ta.rsi(length=24, append=True)
# df.ta.bbands(length=20, std=2, append=True)
# df.ta.macd(fast=15, slow=30, signal=9, append=True)
# df.ta.psar(append=True)

In [27]:
df['ema_short_above_or_below'] = 0
df.loc[(df['EMA_9']>df['EMA_21']), 'ema_short_above_or_below'] = 1
df.loc[(df['EMA_9']<df['EMA_21']), 'ema_short_above_or_below'] = -1

In [28]:
df['r_close_bbu'] = df['close'] / df['BBU_20_2.0']
df['r_close_bbl'] = df['close'] / df['BBL_20_2.0']
df['r_ema_bbu'] = df['EMA_9'] / df['BBU_20_2.0']
df['r_ema_bbl'] = df['EMA_9'] / df['BBL_20_2.0']

In [29]:
df['long_entry'] = np.nan
df['short_entry'] = np.nan
df['long_exit'] = np.nan
df['short_exit'] = np.nan

In [30]:
display(df.iloc[:30])

,date,open,high,low,close,volume,idx_int,EMA_9,EMA_21,RSI_7,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_6_13_5,MACDh_6_13_5,MACDs_6_13_5,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,psar_flip_dir,r_ema_s_m,gradient_ema_9,gradient_rsi_7,gradient_ls,flag_ema_crossing,flag_grad_ema,flag_grad_rsi,flag_grad_ls,ema_short_above_or_below,r_close_bbu,r_close_bbl,r_ema_bbu,r_ema_bbl,long_entry,short_entry,long_exit,short_exit,ema_crossing_pos,macd_pos,close_to_bbl,rsi_30_to_50,PSAR_bellow_close
0,2024-11-08 10:53:00,2920.67,2922.81,2920.65,2922.79,29.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,2024-11-08 10:54:00,2922.48,2924.93,2922.38,2924.21,51.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2914.930000,NaN,0.04,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
2,2024-11-08 10:55:00,2924.17,2925.30,2923.91,2925.17,31.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2915.330000,NaN,0.06,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
3,2024-11-08 10:56:00,2925.18,2930.20,2925.18,2928.75,89.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2915.928200,NaN,0.08,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
4,2024-11-08 10:57:00,2928.69,2934.19,2927.92,2932.64,221.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2917.069944,NaN,0.10,0,0,NaN,0.000000,0.000000,2.424,0,0,-1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
5,2024-11-08 10:58:00,2932.70,2932.71,2929.84,2929.84,199.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2918.781950,NaN,0.10,0,0,NaN,0.000000,0.000000,1.873,0,0,-1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
6,2024-11-08 10:59:00,2929.67,2930.03,2924.29,2926.59,109.0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2920.322755,NaN,0.10,0,0,NaN,0.000000,0.000000,0.393,0,0,-1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
7,2024-11-08 11:00:00,2926.79,2928.45,2926.37,2927.67,25.0,7,NaN,NaN,57.189145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2921.709479,NaN,0.10,0,0,NaN,0.000000,11.437829,-0.821,0,0,1,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
8,2024-11-08 11:01:00,2927.73,2929.71,2924.13,2924.29,70.0,8,2926.883333,NaN,42.406600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2922.957531,NaN,0.10,0,0,NaN,585.376667,14.200235,-1.887,0,1,1,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,1
9,2024-11-08 11:02:00,2924.34,2926.26,2923.37,2925.82,41.0,9,2926.670667,NaN,49.324232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2934.190000,0.02,1,-1,NaN,878.022467,14.105506,-1.034,0,1,1,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0


# Strategy

With access to real-time crypto data on a 1-minute frequency, you can use a high-frequency trading strategy that relies on these indicators to capture quick, small movements. Here’s a 1-minute crypto trading strategy using Bollinger Bands, MACD, RSI, Parabolic SAR, and EMA.

### Strategy Overview

In this strategy:
- **Indicators**:
  - **Bollinger Bands (BB)**: Used for spotting volatility squeezes and reversal points.
  - **MACD**: Helps to confirm the trend and momentum.
  - **RSI**: Identifies overbought and oversold conditions.
  - **Parabolic SAR**: Shows potential reversal points.
  - **EMA**: Provides trend direction, but with short-period settings for 1-minute data.

### Indicator Setup

1. **EMA**: 9-period (short-term trend) and 21-period (medium-term trend) EMAs to define trend direction.
2. **Bollinger Bands**: 20-period with 2 standard deviations, adapted for quick reversals and price channels.
3. **MACD**: Use fast settings, such as 6-period for the short EMA, 13-period for the long EMA, and a 5-period signal line, to react to short-term price movements.
4. **RSI**: 7-period RSI to make it more sensitive to overbought/oversold conditions in a 1-minute context.
5. **Parabolic SAR**: Default settings (with high sensitivity) for trend reversal signals.

### Trading Rules

#### Long Entry Signal

1. **Confirm the Trend**:
   - **EMA Crossover**: The 9 EMA crosses above the 21 EMA, indicating a potential short-term uptrend.
   - **MACD**: MACD histogram or line is positive, showing bullish momentum.

2. **Entry Conditions**:
   - **Bollinger Bands**: Price touches or briefly dips below the lower Bollinger Band.
   - **RSI**: RSI is between 30 and 50, indicating the price is emerging from an oversold state.
   - **Parabolic SAR**: Ensure Parabolic SAR dots are below the price, suggesting an upward trend.

3. **Enter Long Position**: When all conditions are met, enter a long position.

#### Short Entry Signal

1. **Confirm the Trend**:
   - **EMA Crossover**: The 9 EMA crosses below the 21 EMA, suggesting a potential short-term downtrend.
   - **MACD**: MACD histogram or line is negative, showing bearish momentum.

2. **Entry Conditions**:
   - **Bollinger Bands**: Price touches or slightly breaches the upper Bollinger Band.
   - **RSI**: RSI is between 50 and 70, indicating overbought conditions.
   - **Parabolic SAR**: Dots appear above the price, indicating a downward trend.

3. **Enter Short Position**: When all conditions align, enter a short position.

#### Exit Conditions

1. **Long Exit**:
   - **Bollinger Bands**: Price reaches the upper Bollinger Band or moves near it.
   - **RSI**: RSI rises above 70, showing an overbought condition.
   - **Parabolic SAR**: Dots flip to above the price, signaling a potential end of the uptrend.

2. **Short Exit**:
   - **Bollinger Bands**: Price reaches or approaches the lower Bollinger Band.
   - **RSI**: RSI falls below 30, showing an oversold condition.
   - **Parabolic SAR**: Dots flip below the price, indicating a potential end of the downtrend.

#### Stop-Loss and Take-Profit Levels

1. **Stop-Loss**: Place a stop-loss order just outside the recent swing low (for long trades) or swing high (for short trades), typically around 0.2-0.3% for 1-minute data.
2. **Take-Profit**: Set a target based on Bollinger Band levels or RSI extremes, aiming for a quick profit margin of 0.5-1%.

### Example of a 1-Minute Trade

#### Bullish Setup

1. **Trend Confirmation**: 9 EMA crosses above 21 EMA, MACD is positive.
2. **Entry Signal**: Price touches the lower Bollinger Band, RSI is near 40, Parabolic SAR dots appear below price.
3. **Entry**: Enter a long position.
4. **Exit**: Price reaches the upper Bollinger Band, RSI is above 70, or Parabolic SAR dots appear above the price.

#### Bearish Setup

1. **Trend Confirmation**: 9 EMA crosses below 21 EMA, MACD is negative.
2. **Entry Signal**: Price touches the upper Bollinger Band, RSI is near 60, Parabolic SAR dots appear above price.
3. **Entry**: Enter a short position.
4. **Exit**: Price reaches the lower Bollinger Band, RSI falls below 30, or Parabolic SAR dots appear below the price.

### Tips for Real-Time Trading

1. **Use Alerts**: Set up alerts for EMA crossovers, Bollinger Band touches, and Parabolic SAR reversals to help react quickly to signals.
2. **Backtest**: Backtest the strategy on historical 1-minute crypto data to fine-tune parameters based on volatility and crypto type.
3. **Risk Management**: Use a maximum position size per trade and a loss limit per session to manage risk in high-frequency conditions.

This high-frequency strategy can help capitalize on quick, small moves in the crypto market, taking advantage of the volatility seen in minute-by-minute data.

# Long Entry

In [31]:
mask_le1 = (df['ema_short_above_or_below']==1)&(df['flag_ema_crossing']==1)
mask_le2 = (df['MACD_6_13_5']>0)|(df['MACDh_6_13_5']>0)
mask_le3 = (df['r_close_bbl']<=1.0005)
mask_le4 = (df['RSI_7']<50)&(df['RSI_7']>=30)
mask_le5 = (df['PSARl_0.02_0.2'] < df['close'])

In [32]:
df['ema_crossing_pos'] = 0
df.loc[mask_le1, 'ema_crossing_pos'] = 1
df['macd_pos'] = 0
df.loc[mask_le2, 'macd_pos'] = 1
df['close_to_bbl'] = 0
df.loc[mask_le3, 'close_to_bbl'] = 1
df['rsi_30_to_50'] = 0
df.loc[mask_le4, 'rsi_30_to_50'] = 1
df['PSAR_bellow_close'] = 0
df.loc[mask_le5, 'PSAR_bellow_close'] = 1

In [33]:
df['long_entry'] = df[['ema_crossing_pos', 'macd_pos', 'close_to_bbl', 'rsi_30_to_50', 'PSAR_bellow_close']].sum(axis=1)

In [35]:
df.loc[df['long_entry'] == 4]

,date,open,high,low,close,volume,idx_int,EMA_9,EMA_21,RSI_7,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_6_13_5,MACDh_6_13_5,MACDs_6_13_5,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,psar_flip_dir,r_ema_s_m,gradient_ema_9,gradient_rsi_7,gradient_ls,flag_ema_crossing,flag_grad_ema,flag_grad_rsi,flag_grad_ls,ema_short_above_or_below,r_close_bbu,r_close_bbl,r_ema_bbu,r_ema_bbl,long_entry,short_entry,long_exit,short_exit,ema_crossing_pos,macd_pos,close_to_bbl,rsi_30_to_50,PSAR_bellow_close
50,2024-11-08 11:43:00,2926.14,2926.15,2922.22,2922.44,82.0,50,2923.767931,2923.412138,43.761029,2919.268395,2922.8810,2926.493605,0.247195,0.438964,0.582157,0.052206,0.529951,2920.814826,NaN,0.04,0,0,1.000122,0.471843,-3.517298,-0.013,1,1,-1,0,1,0.998615,1.001086,0.999069,1.001541,4,NaN,NaN,NaN,1,1,0,1,1
51,2024-11-08 11:44:00,2922.29,2922.98,2922.23,2922.23,16.0,51,2923.460345,2923.304671,42.707571,2919.244056,2922.7500,2926.255944,0.239907,0.425840,0.233858,-0.197395,0.431253,2921.065833,NaN,0.04,0,0,1.000053,0.171275,-9.683692,-1.031,1,1,-1,-1,1,0.998624,1.001023,0.999045,1.001444,4,NaN,NaN,NaN,1,1,0,1,1
52,2024-11-08 11:45:00,2922.23,2923.02,2922.22,2922.42,30.0,52,2923.252276,2923.224247,44.127316,2919.215833,2922.6625,2926.109167,0.235858,0.464821,0.038212,-0.262027,0.300240,2921.306799,NaN,0.04,0,0,1.000010,-0.128980,-9.999856,-1.330,1,-1,-1,-1,1,0.998739,1.001098,0.999024,1.001383,4,NaN,NaN,NaN,1,1,0,1,1
93,2024-11-08 12:26:00,2920.07,2920.38,2917.99,2918.25,58.0,93,2918.692581,2918.514720,47.878229,2914.110524,2917.6760,2921.241476,0.244405,0.580494,0.319736,-0.021446,0.341183,2914.280167,NaN,0.02,0,0,1.000061,0.172731,1.087743,0.262,1,1,1,1,1,0.998976,1.001420,0.999127,1.001572,4,NaN,NaN,NaN,1,1,0,1,1
127,2024-11-08 13:00:00,2939.39,2939.69,2936.35,2936.42,42.0,127,2937.873337,2935.820532,45.259571,2930.452761,2936.6040,2942.755239,0.418936,0.485044,0.713864,-0.323475,1.037340,2932.696818,NaN,0.04,0,0,1.000699,0.262217,-1.848211,-0.111,1,1,-1,-1,1,0.997847,1.002036,0.998341,1.002532,4,NaN,NaN,NaN,1,1,0,1,1
128,2024-11-08 13:01:00,2936.43,2937.34,2933.87,2937.34,56.0,128,2937.766670,2935.958666,49.936771,2932.796923,2937.1180,2941.439077,0.294239,0.525688,0.507958,-0.352921,0.860879,2933.140145,NaN,0.04,0,0,1.000616,0.091374,-4.051082,-0.592,1,1,-1,-1,1,0.998606,1.001549,0.998751,1.001695,4,NaN,NaN,NaN,1,1,0,1,1
129,2024-11-08 13:02:00,2937.33,2939.70,2936.18,2937.19,84.0,129,2937.651336,2936.070605,49.138139,2934.311742,2937.4460,2940.580258,0.213400,0.459161,0.339731,-0.347432,0.687163,2933.565740,NaN,0.04,0,0,1.000538,-0.088901,-4.688859,-0.810,1,-1,-1,-1,1,0.998847,1.000981,0.999004,1.001138,4,NaN,NaN,NaN,1,1,0,1,1
136,2024-11-08 13:09:00,2940.96,2941.46,2936.08,2936.15,73.0,136,2939.398389,2938.081602,38.752082,2934.852795,2938.7885,2942.724205,0.267845,0.164800,0.247123,-0.523261,0.770383,NaN,2945.12,0.02,1,-1,1.000448,0.090554,-4.252588,-0.934,1,1,-1,-1,1,0.997766,1.000442,0.998870,1.001549,4,NaN,NaN,NaN,1,1,1,1,0
148,2024-11-08 13:21:00,2937.59,2938.86,2936.95,2937.12,55.0,148,2937.428237,2937.167636,48.505658,2931.339435,2937.6770,2944.014565,0.431468,0.456056,0.172886,0.030732,0.142154,2930.644176,NaN,0.04,0,0,1.000089,0.099562,-2.453893,-0.490,1,1,-1,-1,1,0.997658,1.001972,0.997763,1.002077,4,NaN,NaN,NaN,1,1,0,1,1
200,2024-11-08 14:13:00,2950.77,2952.01,2947.74,2948.64,54.0,200,2951.312974,2949.561161,41.834129,2944.905226,2950.7225,2956.539774,0.394295,0.321007,0.255420,-0.790730,1.046149,2947.197437,NaN,0.04,0,0,1.000594,-0.295043,-8.083083,-2.232,1,-1,-1,-1,1,0.997328,1.001268,0.998232,1.002176,4,NaN,NaN,NaN,1,1,0,1,1


# Short Entry

In [ ]:
mask_se1 = (df['ema_short_above_or_below']==-1)&(df['flag_ema_crossing']==1)
mask_se2 = (df['MACD_6_13_5']<0)|(df['MACDh_6_13_5']<0)
mask_se3 = (df['r_close_bbu']>=0.9995)
mask_se4 = (df['RSI_7']>50)&(df['RSI_7']<=70)
mask_se5 = (df['PSARs_0.02_0.2'] > df['close'])

In [ ]:
df['ema_crossing_neg'] = 0
df.loc[mask_se1, 'ema_crossing_neg'] = 1
df['macd_neg'] = 0
df.loc[mask_se2, 'macd_neg'] = 1
df['close_to_bbu'] = 0
df.loc[mask_se3, 'close_to_bbu'] = 1
df['rsi_50_to_70'] = 0
df.loc[mask_se4, 'rsi_50_to_70'] = 1
df['PSAR_above_close'] = 0
df.loc[mask_se5, 'PSAR_above_close'] = 1

In [ ]:
df['short_entry'] = df[['ema_crossing_neg', 'macd_neg', 'close_to_bbu', 'rsi_50_to_70', 'PSAR_above_close']].sum(axis=1)

In [ ]:
df.loc[df['short_entry']==1].head()

# Long Exit

In [ ]:
mask_lex1 = (df['flag_grad_ema'] == -1)
mask_lex2 = (df['RSI_7']>=70)
mask_lex3 = (df['psar_flip_dir'] == -1)

In [ ]:
df['gradient_ema_neg'] = 0
df.loc[mask_lex1, 'gradient_ema_neg'] = 1
df['rsi_above_70'] = 0
df.loc[mask_lex2, 'rsi_above_70'] = 1
df['psar_flip_neg'] = 0
df.loc[mask_lex3, 'psar_flip_neg'] = 1

In [ ]:
df['long_exit'] = df[['gradient_ema_neg', 'rsi_above_70', 'psar_flip_neg']].sum(axis=1)

# Short Exit

In [ ]:
mask_sex1 = (df['flag_grad_ema'] == 1)
mask_sex2 = (df['RSI_7']<=30)
mask_sex3 = (df['psar_flip_dir'] == 1)

In [ ]:
df['gradient_ema_pos'] = 0
df.loc[mask_sex1, 'gradient_ema_pos'] = 1
df['rsi_bellow_30'] = 0
df.loc[mask_sex2, 'rsi_bellow_30'] = 1
df['psar_flip_pos'] = 0
df.loc[mask_sex3, 'psar_flip_pos'] = 1

In [ ]:
df['short_exit'] = df[['gradient_ema_pos', 'rsi_bellow_30', 'psar_flip_pos']].sum(axis=1)

# Bullish / bearish confirmation

In [ ]:
df['bullish_confirm'] = df[['ema_crossing_pos', 'macd_pos', 'gradient_ema_pos']].sum(axis=1)
df['bearish_confirm'] = df[['ema_crossing_neg', 'macd_neg', 'gradient_ema_neg']].sum(axis=1)

In [ ]:
df.head(30)

In [ ]:
df.to_clipboard()